# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,33.05,64,75,2.06,US,1693939459
1,1,chonchi,-42.6128,-73.8092,10.56,96,100,2.92,CL,1693939736
2,2,stanley,54.8680,-1.6985,18.07,83,0,3.09,GB,1693939511
3,3,port-aux-francais,-49.3500,70.2167,2.51,76,100,14.33,TF,1693939494
4,4,mineiros,-17.5694,-52.5511,30.16,38,99,2.39,BR,1693940035


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size="Humidity", scale=0.7, color="City")

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <=30) & (city_data_df["Max Temp"] > 15) &
                                   (city_data_df["Humidity"] < 70)]

# Drop any rows with null values
city_data_df.dropna(axis=0, inplace=True)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,baiyin,36.5583,104.2081,19.05,69,5,2.19,CN,1693940036
10,10,santa elena de uairen,4.6167,-61.1333,27.31,48,87,4.50,VE,1693940039
38,38,okotoks,50.7289,-113.9828,18.77,67,0,1.79,CA,1693939627
47,47,ribeira grande,38.5167,-28.7000,24.22,64,40,11.32,PT,1693939523
49,49,besni,37.6928,37.8611,27.36,29,0,0.93,TR,1693940061
...,...,...,...,...,...,...,...,...,...,...
564,564,santa rosa de viterbo,-21.4728,-47.3631,26.56,63,100,3.42,BR,1693940374
567,567,la carolina,38.2756,-3.6153,24.47,35,100,2.28,ES,1693940375
570,570,wainwright,52.8390,-110.8572,16.09,39,9,4.77,CA,1693940377
583,583,shihezi,44.3000,86.0333,16.78,51,6,1.73,CN,1693940385


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df=hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,baiyin,CN,36.5583,104.2081,69,
10,santa elena de uairen,VE,4.6167,-61.1333,48,
38,okotoks,CA,50.7289,-113.9828,67,
47,ribeira grande,PT,38.5167,-28.7000,64,
49,besni,TR,37.6928,37.8611,29,
...,...,...,...,...,...,...
564,santa rosa de viterbo,BR,-21.4728,-47.3631,63,
567,la carolina,ES,38.2756,-3.6153,35,
570,wainwright,CA,52.8390,-110.8572,39,
583,shihezi,CN,44.3000,86.0333,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit":20}
    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
baiyin - nearest hotel: 红鹭宾馆
santa elena de uairen - nearest hotel: Posada Villa Fairmont
okotoks - nearest hotel: Royal Duke Hotel
ribeira grande - nearest hotel: Hospedaria JSF
besni - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
gaspe - nearest hotel: Hotel Plante
arraial do cabo - nearest hotel: No hotel found
ranchettes - nearest hotel: Staybridge Suites Cheyenne
shalqar - nearest hotel: Бекнур
the pas - nearest hotel: Wescana Inn
khenifra - nearest hotel: Najah
zheleznodorozhnyy - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
ankazoabo - nearest hotel: No hotel found
mejit - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
tamanrasset - nearest hotel: فندق الأمان
mana - nearest hotel: Le Samana
constantia - nearest hotel: Hotel Chérica
altinyayla - nearest hotel: No hotel found
mersa matruh - nearest hotel: صقور
happy valley-goose bay - nearest hotel: Hotel North Two


,City,Country,Lat,Lng,Humidity,Hotel Name
5,baiyin,CN,36.5583,104.2081,69,红鹭宾馆
10,santa elena de uairen,VE,4.6167,-61.1333,48,Posada Villa Fairmont
38,okotoks,CA,50.7289,-113.9828,67,Royal Duke Hotel
47,ribeira grande,PT,38.5167,-28.7000,64,Hospedaria JSF
49,besni,TR,37.6928,37.8611,29,No hotel found
...,...,...,...,...,...,...
564,santa rosa de viterbo,BR,-21.4728,-47.3631,63,Malim
567,la carolina,ES,38.2756,-3.6153,35,La Gran Parada
570,wainwright,CA,52.8390,-110.8572,39,Park Hotel
583,shihezi,CN,44.3000,86.0333,51,石河子宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size = "Humidity", color="City", frame_width = 800, 
                                   frame_height = 400, scale = 0.7, hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)